## Математический вывод ЕМ (Expectation-maximization) алгоритма

Рассмотрим вероятностную модель смеси нормальных распределений (в общем случае может быть рассмотрено другое семейство распределений):

$$p(x) = \sum_{z}p(x, z) = \sum_{k=1}^K w_k N(x | \theta_k) $$

Здесь 

$N(x | \theta_k)$ - плотность некоторого ($k$-ого) распределения из семейства нормальных

$K$ - количество распределений из семейства нормальных в смеси

$z$ - скрытая переменная, $K$-мерный бинарный случайный вектор, ровно одна
компонента которого равна единице (скрытая переменная). Определяет какое именно распределение из множества $K$ в смеси было взято

$w_k$ - приорная вероятность взять то или иное распределение из семейства. Вероятность того, что единице будет равна $k$-я компонента вектора $z$

$x$ - объект из выборки

Рассмотрим вероятностную модель с наблюдаемыми переменными X, параметрами $\theta$ и скрытыми переменными $z$, для которой задано полное правдоподобие $log p(X, Z | \Theta)$:

$$log p(X, Z | \Theta) = \sum_{i=1}^l \sum_{k=1}^K z_{ik} \big\{ log w_k + log N(x_i | \theta_k) \big\} $$
$l$ - количество объектов в выборе X

EM-алгоритм решает задачу максимизации полного правоподобия путем попеременной оптимизации по параметрам и по скрытым переменным.

#### Шаг E
Воспользуемся байесовским подходом, зафиксируем вектор параметров $\Theta^{old}$ и вычислим апостериорное распределение на скрытых переменных $p(Z | X, \Theta^{old})$:

$$p(Z | X, \Theta^{old}) = \frac{p(X, Z | \Theta^{old})}{p(X |\Theta^{old})}$$

Правая часть равенства пропорциональна:

$$\prod_{i=1}^l \prod_{k=1}^K w_k^{old} N (x_i | \mu_k^{old}, \Sigma_k^{old})$$

А данное распределение распадается в произведение распределений, соответствующих отдельным объектам $p(z_i| x_i, \Theta)$:

$$p(Z | X, \Theta^{old}) = \prod_{i=1}^l  p(z_i| x_i, \Theta^{old})$$

Из записанного следует, что величины $z_i$ независимы при известных объектах $x_i$. Вектор $z_i$ имеет K возможных значений. Запишем   все их вероятности, воспользовавшись формулой Байеса:

$$g_{ik} = p(z_{ik}=1 | x_i, \Theta^{old}) =  \frac{p(z_{ik}=1) p(x_i | z_{ik}=1, \Theta^{old})}{\sum_{j=1}^K p(z_{ij}=1) p(x_i | z_{ij}=1, \Theta^{old})}$$



Запишем теперь матожидание логарифма полного правдоподобия по всем возможным значениям
скрытых переменных $Z$ с весами, равными апостериорным вероятностям этих переменных $p(Z | X, \Theta^{old})$:

$$Q(\Theta, \Theta^{old}) = E_{Z} log p(X, Z | \Theta) = E_Z \sum_{i=1}^l \sum_{k=1}^K z_{ik} \big\{log w_k + log N(x_i | \mu_k, \Sigma_k)\big\} = \sum_{i=1}^l \sum_{k=1}^K E_Z [z_{ik}] \big\{log w_k + log N(x_i | \mu_k, \Sigma_k)\big\}  $$

Замтим, что:
$$E_Z [z_{ik}] = 1 * p(z_{ik}=1 | x_i, \Theta) + 0 * p(z_{ik}=0 | x_i, \Theta) = g_{ik}$$

Тогда 

$$Q(\Theta, \Theta^{old}) = \sum_{i=1}^l \sum_{k=1}^K g_{ik} \big\{log w_k + log N(x_i | \mu_k, \Sigma_k)\big\} $$

#### Шаг M
Новый вектор параметров находится как максимизатор матожидания из шага Е по параметрам тета:

$$\Theta^{new} = argmax_{\Theta} E_{Z} log p(X, Z | \Theta) $$

Для этого продифференцируем функционал, полученный на шаге Е (логарифм полного правдоподобия), и затем все частные производные приравняем к 0. Из полученной системы уравнений мы найдем формулы для шага М с учетом ограничения $\sum_{k=1}^K w_k = 1$:

$$\frac{dQ(\Theta, \Theta^{old})}{d w_k} = \frac{1}{w_k} \sum_{i=1}^l g_{ik} - l = 0 $$

$$ w_k = \frac{1}{l} \sum_{i=1}^l g_{ik} $$

И аналогично для матицы ковариаций и центров:

$$\mu_k = \frac{1}{l w_k} \sum_{i=1}^l g_{ik} x_i $$

$$\Sigma_k =  \frac{1}{l w_k} \sum_{i=1}^l g_{ik} (x_i - \mu_k)(x_i - \mu_k)^T$$